In [75]:
from itertools import combinations
import numpy as np
import random

GOAL
Cost minimization

## Simple Test Problem

In [76]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('problems/test_problem.npy')

## Common tests

In [77]:
p_g = np.load('problems/problem_g_100.npy')
p_r1 = np.load('problems/problem_r1_100.npy')
p_r2 = np.load('problems/problem_r2_100.npy')

In [78]:
print(p_g)

[[  0.         179.92038688 176.126132   ... 162.61509016 407.15461186
  141.77638964]
 [179.92038688   0.         278.35892376 ... 116.33000682 281.45211088
  154.17643486]
 [176.126132   278.35892376   0.         ... 178.74068495 375.52755063
  312.44633915]
 ...
 [162.61509016 116.33000682 178.74068495 ...   0.         245.06340681
  224.97614312]
 [407.15461186 281.45211088 375.52755063 ... 245.06340681   0.
  434.61707535]
 [141.77638964 154.17643486 312.44633915 ... 224.97614312 434.61707535
    0.        ]]


In [79]:
print(p_r1)

[[  0.           4.9766      65.54479501 ...   7.83902018  27.41995194
   87.69299895]
 [  9.36499057   0.         110.87932346 ...  19.63789468  30.94641048
   50.11884753]
 [ 35.88566785  54.64069406   0.         ...  66.45444707  87.16585662
   13.71356104]
 ...
 [ 15.7861903   13.58665954  37.36828725 ...   0.          33.44981139
   28.40778893]
 [ 43.48793718  32.44459687  80.22882128 ...  18.97247219   0.
   68.05734486]
 [ 38.95399487  52.21881804  12.94487068 ...  67.75221274  60.95591025
    0.        ]]


In [80]:
print(p_r2)

[[ -9.15367559  47.74344577  40.16586609 ... -40.84436694  49.89304781
   -3.80798436]
 [-49.59100483 -49.17049366 -13.77660833 ...   9.24571587 -10.78074211
  -31.92741522]
 [ 44.58858581  13.16415431  24.0761291  ...  46.47632671 -22.10624941
  -20.34470381]
 ...
 [  1.09354585   9.06381007  16.15032321 ...  32.66745074  41.05556601
   35.1475426 ]
 [ 47.09316012  17.95291657   8.44268749 ...  33.69341475  -3.4474984
  -17.256382  ]
 [ -5.79703165  -5.79169305 -12.96217049 ...  11.97955136  42.27871638
    7.40063403]]


In [81]:
def check_negative_value(problem):
    # Negative values?
    return np.any(problem < 0)


def check_diagonal_all_zero(problem):
    # Diagonal is all zero?
    return np.allclose(np.diag(problem), 0.0)


def check_symmetry(problem):
    # Symmetric matrix?
    return np.allclose(problem, problem.T)


def check_triangular_inequality(problem):
    # Triangular inequality
    return all(
        problem[x, y] <= problem[x, z] + problem[z, y]
        for x, y, z in list(combinations(range(problem.shape[0]), 3))
    )

In [82]:
print(f"check negative value | p_g: {check_negative_value(p_g)}")
print(f"check diagonal all 0s | p_g: {check_diagonal_all_zero(p_g)}")
print(f"check symmetry | p_g: {check_symmetry(p_g)}")
print(f"check triangular inequality | p_g {check_triangular_inequality(p_g)}")
print()
print(f"check negative value | p_r1: {check_negative_value(p_r1)}")
print(f"check diagonal all 0s | p_r1: {check_diagonal_all_zero(p_r1)}")
print(f"check symmetry | p_r1: {check_symmetry(p_r1)}")
print(f"check triangular inequality | p_r1 {check_triangular_inequality(p_r1)}")
print()
print(f"check negative value | p_r2: {check_negative_value(p_r2)}")
print(f"check diagonal all 0s | p_r2: {check_diagonal_all_zero(p_r2)}")
print(f"check symmetry | p_r2: {check_symmetry(p_r2)}")
print(f"check triangular inequality | p_r2 {check_triangular_inequality(p_r2)}")
print()

check negative value | p_g: False
check diagonal all 0s | p_g: True
check symmetry | p_g: True
check triangular inequality | p_g True

check negative value | p_r1: False
check diagonal all 0s | p_r1: True
check symmetry | p_r1: False
check triangular inequality | p_r1 False

check negative value | p_r2: True
check diagonal all 0s | p_r2: False
check symmetry | p_r2: False
check triangular inequality | p_r2 False



In [83]:
# HYPERPARAMETERS

POPULATION_SIZE = 100
NUM_GENERATIONS = 1000
ELITISM_SIZE = 2
TOURNAMENT_K = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2

In [84]:
def create_initial_population(pop_size, num_cities):
    """Creates a list of random tours (solutions)."""
    population = []
    for _ in range(pop_size):
        tour = np.random.permutation(num_cities)
        population.append(tour)
    return population

In [85]:
def calculate_cost(tour, distance_matrix):
    """
    Calculates the total cost (distance) of a single tour.
    This function works for BOTH symmetric and asymmetric matrices
    because it respects the order of the tour.
    """
    total_cost = 0
    num_cities = len(tour)
    for i in range(num_cities):
        city_a = tour[i]
        # Use modulo to consider going back to the starting city
        city_b = tour[(i + 1) % num_cities]
        
        # it takes the cost from A -> B
        total_cost += distance_matrix[city_a, city_b]
        
    return total_cost

In [86]:
def calculate_fitness(tour, distance_matrix):
    """
    Calculates fitness. Since we want to MINIMIZE cost,
    a good fitness function is 1 / cost.
    (This assumes cost is always > 0, which is true for p_g and p_r1)
    """
    cost = calculate_cost(tour, distance_matrix)
    return 1.0 / (cost + 1e-6) # the second term on the denominator has been added for avoiding divisions by 0s.

In [87]:
def tournament_selection(population, fitness_scores, k):
    """
    Selects one parent using k-tournament selection.
    Picks k random individuals and returns the fittest one.
    """
    indices = np.random.choice(len(population), k, replace=False)
    k_fitnesses = [fitness_scores[i] for i in indices]
    best_local_idx = np.argmax(k_fitnesses)
    best_global_idx = indices[best_local_idx]
    
    return population[best_global_idx]

In [88]:
def crossover(parent1, parent2):
    """
    Perform crossover
    """
    num_cities = len(parent1)
    offspring = np.full(num_cities, -1)
    
    # Select two random cut points
    start, end = sorted(np.random.choice(num_cities, 2, replace=False))
    
    # Copy the slice from parent1
    offspring[start:end] = parent1[start:end]
    
    # Fill remaining slots from parent2
    parent2_ptr = 0
    offspring_ptr = 0
    cities_in_offspring = set(offspring[start:end])
    
    while -1 in offspring:
        # Find next available slot in offspring
        if offspring_ptr == start:
            offspring_ptr = end
            
        # Get city from parent2
        city_to_add = parent2[parent2_ptr]
        parent2_ptr += 1
        
        if city_to_add not in cities_in_offspring:
            offspring[offspring_ptr] = city_to_add
            offspring_ptr += 1
            
    return offspring

In [89]:
# DIFFERENT KINDS OF MUTATIONS


def swap_mutation(tour):
    """Performs a simple swap mutation on a tour (modifies in-place)."""
    idx1, idx2 = np.random.choice(len(tour), 2, replace=False)
    tour[idx1], tour[idx2] = tour[idx2], tour[idx1]


def inversion_mutation(tour):
    """
    Performs an inversion mutation (modifies in-place).
    """
    # Select two random cut points
    start, end = sorted(np.random.choice(len(tour), 2, replace=False))
    
    # Reverse the sub-sequence in-place
    tour[start:end] = tour[start:end][::-1]

In [90]:
def algo_1(distance_matrix, pop_size, num_generations, elitism_size, k, crossover_rate, mutation_rate):
    """
    Runs the main evolutionary algorithm loop.
    """
    num_cities = distance_matrix.shape[0]
    
    # Initializing
    population = create_initial_population(pop_size, num_cities)
    best_tour_so_far = None
    best_cost_so_far = float('inf')
    
    
    for gen in range(num_generations):
        # Calculate fitness for the entire population
        fitness_scores = [calculate_fitness(tour, distance_matrix) for tour in population]
        
        # Find the indices of the N best individuals,
        # The last index is the one that refers to the individual with the best fitness
        elite_indices = np.argsort(fitness_scores)[-elitism_size:]
        # creating a new population composed of the best individuals of the previous
        new_population = [population[i].copy() for i in elite_indices]
        
        # Track the best-ever solution
        best_current_idx = elite_indices[-1]
        current_best_cost = calculate_cost(population[best_current_idx], distance_matrix)
        
        if current_best_cost < best_cost_so_far:
            best_cost_so_far = current_best_cost
            best_tour_so_far = population[best_current_idx].copy()
            if gen % 20 == 0: # Print update, but not too often
                print(f"Gen {gen}: New best cost = {best_cost_so_far:.2f}")

        # In this loop we compute the generation
        while len(new_population) < pop_size:
            # Selecting 2 parents using tournament selection
            parent1 = tournament_selection(population, fitness_scores, k)
            parent2 = tournament_selection(population, fitness_scores, k)
            
            # performing the cross over (random decision defined by crossover_rate)
            if random.random() < crossover_rate:
                offspring = crossover(parent1, parent2)
            else:
                offspring = parent1.copy() 
            
            # performing the mutation
            # (again, its a random decision that doesn't even depend on whether the crossover is done or not)
            if random.random() < mutation_rate:
                # Here e we can choose to use inversion_mutation() and swap_mutation()
                inversion_mutation(offspring)
                
            new_population.append(offspring)

        # Replace the old population with the new one
        population = new_population

    return best_tour_so_far, best_cost_so_far

In [91]:
best_tour_p_g, best_cost_p_g = algo_1(
        distance_matrix=p_g,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
    )

print(f"Best tour (p_g): {best_tour_p_g}")
print(f"Total cost (p_g): {best_cost_p_g:.2f}\n")

Gen 0: New best cost = 23134.34
Gen 20: New best cost = 17661.25
Gen 40: New best cost = 15308.21
Gen 100: New best cost = 11049.17
Gen 140: New best cost = 9575.72
Gen 160: New best cost = 9285.16
Gen 180: New best cost = 8489.54
Gen 260: New best cost = 7460.57
Gen 300: New best cost = 6993.32
Gen 340: New best cost = 6573.21
Gen 360: New best cost = 6350.41
Gen 440: New best cost = 5726.34
Gen 540: New best cost = 5358.00
Gen 620: New best cost = 4940.43
Best tour (p_g): [78 88 43 87 86 55 34  6 85 89 69 22  5 37  7 49  1  8  4 75 36 46 93 99
 20 27 63 47 71 74  0 14 41 33 54 67 32 42 81 30 97 64 10 48 31 72 18 56
 70 58  2 11 57 60 21 28 51  3 84 66 73 45 53 16 94 79 44 17 35 38 95 61
 23 82 59  9 62 90 13 76 29 65 39 98 96 25 80 19 40 24 50 26 52 68 83 77
 15 91 12 92]
Total cost (p_g): 4553.08



In [92]:
best_tour_p_r1, best_cost_p_r1 = algo_1(
        distance_matrix=p_r1,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
)

print(f"Best tour (p_g): {best_tour_p_r1}")
print(f"Total cost (p_g): {best_cost_p_r1:.2f}\n")

Gen 0: New best cost = 4657.52
Gen 20: New best cost = 3251.88
Gen 40: New best cost = 2741.64
Gen 60: New best cost = 2389.47
Gen 240: New best cost = 2048.99
Gen 320: New best cost = 1981.94
Gen 560: New best cost = 1892.97
Best tour (p_g): [90 38 54 66 49 21 64 29 43 24 47 65 14 89 18 91  5 42 72  7 96 10 68 50
 35 40 73 26 69 83 78 41 25 62 28 15 95 74 93 81 60 20 77 87 58  1  0 94
 23 22 51 12 13 75 30  8 99 46 55 70 56 71 98 59 92 76 63 61 39  3  6 67
 33 31 82 97 34 37 45 57 19 84 86 52 11 79  2  9 16 32  4 85 17 80 53 36
 88 44 27 48]
Total cost (p_g): 1827.13



In [94]:
best_tour_p_r2, best_cost_p_r2 = algo_1(
        distance_matrix=p_r2,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
)

print(f"Best tour (p_g): {best_tour_p_r2}")
print(f"Total cost (p_g): {best_cost_p_r2:.2f}\n")

Gen 0: New best cost = 16.05
Best tour (p_g): [13 40 37 88 60 39 79 85 24 25 97 27 41 99 64 76 62 86 20 80 48 43 94 96
 19 98 70 14 42 44 78 68 16  9 18 95 10  8 28 56 82 57  2 34 51 11  3 49
 30 58 92 46 50 21 65 52 71 22 61 90  1 55 67 75 17 15 72 84 74 12  4 23
 69 83 38  6 32 59 47 45 35 31  5 54 91 53 87 77  7 26 29 36 73 89 66 93
  0 63 81 33]
Total cost (p_g): 0.06

